In [39]:
import numpy as np
import pickle
import math
from pennylane.labs import resource_estimation as re
from utils import *
from pennylane.labs.trotter_error import vibronic_fragments

# Resource Estimation for Quantum Algorithm for Vibronic Dynamics

The following cells define the quantum circuits that implement the different parts of the vibronic Hamiltonian:

#### Circuit for implementing linear ($Q_r$) terms:

This circuit, `Q_cir`, is designed to implement the potential energy terms that are linear in the position operator, $Q_r$. The core logic involves loading coefficients from a QROM and then using a controlled-addition operation to apply a phase proportional to the mode's value onto a phase gradient register. The circuit uncomputes the loaded coefficients at the end to free the ancilla qubits.

In [3]:
def Q_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    for i in range(k):
        ctrl_wire = [mode_wires[i]]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires+ctrl_wire)

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))
     
    re.ResourceIdentity(wires = scratch_wires + total_mode_wires[k:])

#### Circuit for implementing quadratic ($Q_r^2$) terms:

The `Qsquared_cir` function implements the potential energy terms that are quadratic in the displacement operator, $Q_r^2$. Similar to the linear case, it begins by loading coefficients from a QROM. However, it then uses an `OutOfPlaceSquare` routine to compute the square of the mode register's value into a separate scratch register. This squared value is then used to control the phase addition. Finally, the circuit uncomputes the squared value and the loaded coefficients.

In [4]:
def Qsquared_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires)

    for i in range(2*k):
        ctrl_wire = [f"s_{i}"]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

    re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=scratch_wires+mode_wires))

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))

    re.ResourceIdentity(wires = total_mode_wires[k:])

#### Circuit for implementing Bilinear ($Q_r Q_s$) terms:

This circuit, `QrQs_cir`, handles the potential energy terms that involve the product of two distinct vibrational mode operators, $Q_r Q_s$. It uses an `OutMultiplier` circuit to compute the product of two different mode registers into a scratch register. This product then controls the phase addition operation, simulating the evolution for the bilinear term. The circuit concludes by uncomputing the product and the coefficients loaded from the QROM.

In [5]:
def QrQs_cir(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, mode2_wires, scratch_wires):
    re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires)

    re.ResourceOutMultiplier(a_num_qubits=k, b_num_qubits=k, wires=scratch_wires+mode_wires+mode2_wires)

    for i in range(2*k):
        ctrl_wire = [f"s_{i}"]
        target_wires = coeff_wires[:b-i] + phase_grad_wires[:b-i]
        re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - i), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

    re.ResourceAdjoint(re.ResourceOutMultiplier(a_num_qubits=k, b_num_qubits=k, wires=scratch_wires+mode_wires+mode2_wires))

    re.ResourceAdjoint(re.ResourceQROM(num_bitstrings=N, size_bitstring=b, clean=False, wires=electronic_wires + coeff_wires))

    re.ResourceIdentity(wires = total_mode_wires[2*k:])

#### Changing the default resource decomposition for QFT:

By default, PennyLane's resource estimation module uses a textbook implementation of the Quantum Fourier Transform (QFT). Here, we replace it with a more resource-efficient decomposition based on the methods presented in the paper at `https://arxiv.org/abs/1803.04933v2`. The `AQFT_resource_decomp` function calculates the Toffoli gate and auxiliary qubit counts based on the improved algorithm. We then use the `set_resources` method to make this efficient QFT implementation the new default for all subsequent resource estimations within this notebook.

In [6]:
def AQFT_resource_decomp(num_wires, **kwargs):
    ceil_log_n = math.ceil(math.log2(num_wires))
    aux_qubit_count = num_wires + 3*ceil_log_n - 4
    
    toff = re.ResourceToffoli.resource_rep()
    toff_count = 2 * num_wires*(ceil_log_n - 1)

    gate_list = [
        re.AllocWires(aux_qubit_count),
        re.GateCount(toff, toff_count),
        re.FreeWires(aux_qubit_count),
    ]
    return gate_list

re.ResourceQFT.set_resources(AQFT_resource_decomp)
# re.ResourceQFT.set_resources(re.ResourceQFT.default_resource_decomp)  # reset to default QFT cost

#### Circuit for applying a Unitary rotation on the electronic subspace:

This block defines the `digonalizing_circuit`, which represents a general unitary transformation applied to the electronic state qubits. The cost is estimated using the `ResourceQubitUnitary` block, which models the synthesis of an arbitrary unitary operation to a given precision.

In [7]:
re.ResourceMultiplexer.set_resources(re.ResourceMultiplexer.phase_grad_resource_decomp)

def digonalizing_circuit(log_N, electronic_wires):
    re.ResourceQubitUnitary(num_wires=log_N, precision=1e-6, wires=electronic_wires)

##### Circuit for Implementing the Kinetic Fragment

This circuit, `kinetic_cir`, implements the kinetic energy portion of the Hamiltonian, which involves momentum operators squared ($P_r^2$). Since the momentum operator is diagonal in the momentum basis, the circuit first uses a Quantum Fourier Transform (QFT) to each vibrational mode register to switch from the position basis to the momentum basis.

Once in the momentum basis, the circuit squares the value in each mode register, similar to the implementation of the $Q_r^2$ terms. This squared momentum value then controls a phase addition to simulate the kinetic evolution. Finally, an inverse QFT transforms the registers back to the position basis, completing the operation.

In [8]:
def kinetic_cir(b, M, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires):
    for i in range(M):
        re.ResourceQFT(num_wires=k, wires=total_mode_wires[i*k: k*(i + 1)])

    for i in range(M):
        mode_wires = total_mode_wires[i*k: k*(i + 1)]
        re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires)

        for j in range(2*k):
            ctrl_wire = [f"s_{j}"]
            target_wires = coeff_wires[:b-j] + phase_grad_wires[:b-j]
            re.ResourceControlled(re.ResourceSemiAdder(max_register_size=b - j), num_ctrl_wires=1, num_ctrl_values=0, wires=target_wires + ctrl_wire)

        re.ResourceAdjoint(re.ResourceOutOfPlaceSquare(register_size=k, wires=mode_wires+scratch_wires))

    for i in range(M):
        re.ResourceAdjoint(re.ResourceQFT(num_wires=k), wires=total_mode_wires[i*k: k*(i + 1)])

    re.ResourceIdentity(wires=electronic_wires)

## Resource Estimation for Systems of Interest

#### Loading the Hamiltonian:

This code cell loads the Hamiltonian parameters for the molecule of interest. These parameters include the vibrational mode frequencies (`omegas`) and the various coupling strengths (`lambdas`, `alphas`, `betas`) that define the interactions between electronic states and vibrational modes. The script extracts the number of electronic states (`N`) and vibrational modes (`M`) and organizes the coupling terms, setting up the specific problem instance for resource estimation.

In [ ]:
''' 
Uncomment the molecule that resource estimation should be performed for.
'''

# mol = 'no4a_monomer' 
# mol = 'no4a_dimer'            
# mol = 'anthra-c60_ct'
# mol = 'anthracene_6s_66m'
# mol = 'pentacene_16s_102m'
mol = 'maleimide_5s_24m_bilin'
# mol = 'maleimide_5s_24m_nobilin'

k = 4 # Number of Qubits for discretization of each mode
b = 20 # Number of Qubit used for the resource state for the phase gradient operation

filehandler = open(f'model_params/{mol}.pkl', 'rb')
omegas, couplings = pickle.load(filehandler)
filehandler.close()

QVC = bool(2 in couplings)
# Check if the model has contain Quadratic Couplings
omegas = np.array(omegas)
lambdas = couplings[0]
alphas = couplings[1]

N, M = alphas.shape[0], alphas.shape[2]
log_N = math.ceil(math.log2(N))

if QVC:
    betas = couplings[2]
else:
    betas = np.zeros((N, N, M, M))

#### Naming the wires used in the circuits:

To maintain clarity and correctly map circuit operations to the appropriate registers, this cell defines names for the different sets of wires (qubits).

In [10]:
phase_grad_wires = [f"pg_{i}" for i in range(b)]
electronic_wires = [f"e_{i}" for i in range(log_N)]
coeff_wires = [f"c_{i}" for i in range(b)]
total_mode_wires = [f"m_{i}" for i in range(k*M)]
mode_wires = total_mode_wires[:k]
scratch_wires = [f"s_{i}" for i in range(2*k)] 

my_gs = {'X', 'Z', 'Y', 'S', 'Hadamard', 'CNOT', 'RZ', 'Toffoli'}

#### Estimating the cost of implementing linear ($Q_r$) terms for the given system size:

In [11]:
res_Q = re.estimate_resources(Q_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)
print_Toff(res_Q)

Toffoli Count = 146


#### Similarly, estimating the cost of implementing quadratic ($Q_r^2$) terms:

In [12]:
res_Qsquared = re.estimate_resources(Qsquared_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)
print_Toff(res_Qsquared)

Toffoli Count = 272


#### Next, estimating the cost of implementing bilinear ($Q_r Q_s$) terms:

In [13]:
mode2_wires = total_mode_wires[k:2*k]

res_QrQs = re.estimate_resources(QrQs_cir, gate_set=my_gs)(b, N, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, mode2_wires, scratch_wires)
print_Toff(res_QrQs)

Toffoli Count = 310


### Fragmention of Hamiltonian (OG)

This cell uses the "Original Grouping" (OG) method to decompose the hamiltonian into fast-forwardable fragments, as described in the Narrative document.

In [14]:
fragments_OG = vibronic_fragments(N, M, omegas, [lambdas, alphas, betas])
print(f'Decomposed Hamiltonian into {len(fragments_OG)} fragments using the Orginal Grouping(OG) method')

Decomposed Hamiltonian into 9 fragments using the Orginal Grouping(OG) method


#### Estimating the cost of implementing the kinetic fragment:

In [15]:
res_kinetic = re.estimate_resources(kinetic_cir, gate_set=my_gs)(b, M, k, phase_grad_wires, electronic_wires, coeff_wires, total_mode_wires, mode_wires, scratch_wires)

Qubit_count = res_kinetic.qubit_manager.total_qubits
print_Qubit_Toff(res_kinetic)

Qubit Count = 166
Toffoli Count = 6768


#### Counting Terms in Each Fragment

To calculate the total cost of simulating a potential energy fragment, we must first determine its composition. The `count_nonzero_Q_terms` utility function is used for this purpose; it inspects a given fragment and returns the number of linear ($Q_r$), quadratic ($Q_r^2$), and bilinear ($Q_r Q_s$) terms it contains. The subsequent loop then computes the total cost for each fragment by summing the costs of its constituent parts, using the pre-calculated resource estimates for `res_Q`, `res_Qsquared`, and `res_QrQs`.

In [16]:
res_list_OG = [res_kinetic]

for i in range(len(fragments_OG)-1):
    num_Qr, num_Qr2, num_QrQs = count_nonzero_Q_terms(fragments_OG[i])
    res_frag = num_Qr * res_Q + num_Qr2 * res_Qsquared + num_QrQs * res_QrQs
    res_list_OG.append(res_frag)

#### Single Trotter Step Cost (OG)

This section calculates the total resource cost for a single step of the simulation using a second-order Trotter product formula. For efficiency, the two most expensive fragments (those with the highest Toffoli count) are applied once per step, while all other, less costly fragments are applied twice. The code implements this logic to sum the fragment costs and arrive at the total cost for one Trotter step.

In [17]:
res_expensive_frags = sorted(res_list_OG, key=lambda res_frag: res_frag.clean_gate_counts.get("Toffoli", 0),reverse=True)[:2]
step_OG_res = res_expensive_frags[0] + res_expensive_frags[1] 

for res in res_list_OG:
    if res not in res_expensive_frags:
        step_OG_res += res * 2

print_Qubit_Toff(step_OG_res)

Qubit Count = 166
Toffoli Count = 3.158E+04


### Fragmentation of Hamiltonian (Mode Based)

This section decomposes the Hamiltonian based on the "mode-based" fragmentation method described in the Narrative Document.

In [18]:
fragments_mode = vibronic_fragments_modebased(N, M, omegas, [lambdas, alphas, betas], 'FC')

num_frags_mode = len(fragments_mode)
print(f'Decomposed Hamiltonian into {num_frags_mode} fragments using the mode based fragmentation method')

Decomposed Hamiltonian into 22 fragments using the mode based fragmentation method


#### Summing up Cost of Each Fragment 

Similar to the OG section, counting position operator product terms in each fragment and summing up the cost of each fragment.

In [19]:
res_list_mode = [res_kinetic]

for i in range(len(fragments_mode)-1):
    num_Qr, num_Qr2, num_QrQs = count_nonzero_Q_terms(fragments_mode[i])
    res_frag = num_Qr * res_Q + num_Qr2 * res_Qsquared + num_QrQs * res_QrQs
    res_list_mode.append(res_frag)

res_expensive_frags = sorted(res_list_mode, key=lambda res_frag: res_frag.clean_gate_counts.get("Toffoli", 0),reverse=True)[:2]
step_mode_res = res_expensive_frags[0] + res_expensive_frags[1] 

for res in res_list_mode:
    if res not in res_expensive_frags:
        step_mode_res += res * 2

#### Adding the cost of electronic unitaries used for diagonalization:

In [20]:
res_unitary = re.estimate_resources(digonalizing_circuit, gate_set=my_gs,)(log_N, electronic_wires)

Rz_count = res_unitary.clean_gate_counts.get("RZ", 0)
res_RZ = re.estimate_resources(re.ResourceControlled(re.ResourceSemiAdder(abs(math.floor(math.log2(1e-6)))), 1, 0))
res_unitary += Rz_count*res_RZ # Synthesizing each RZ rotation using a adder to the Resource state

#### Single Step Cost (Mode Based)

In [21]:
step_mode_res += res_unitary 
print_Qubit_Toff(step_mode_res)

Qubit Count = 166
Toffoli Count = 2.656E+04


### Number of Trotter Steps

The overall accuracy of a Trotter-based quantum simulation is determined by the number of discrete time steps used. This section outlines the process for calculating how many steps are needed to achieve a target error tolerance for a given total simulation time.

#### Loading the Spectral Norm of the Trotter Error Operator

The spectral norm of the Trotter error operator, $\lambda$, is loaded from a CSV file for the given system.

In [25]:
norm_OG = get_norm_value(mol, 2**k, M, fragmentation_scheme='OG')
norm_mode = get_norm_value(mol, 2**k, M, fragmentation_scheme='mode')

### Calculating the Required Number of Trotter Steps

The required number of trotter steps n, for a simulation for time t, and an error tolerance of $\epsilon$ is:

$$
n = t^{1.5}\cdot \sqrt{ \frac{\lambda}{\epsilon}}.
$$

In [32]:
def num_steps(norm, req_error, total_time):
    return math.ceil(total_time**1.5 * (norm / req_error)**0.5)

total_time = 152 # in ev, time that we want to get the cost for (100 fs).
req_error = 0.01 # 1 percent error

if norm_OG is not None:
    nsteps_OG = num_steps(norm_OG, req_error, total_time)
    print(f'{nsteps_OG} Trotter steps are required for a {100*req_error}% error tolerance in simulation of the hamiltonian using the OG fragmentation scheme.')

if norm_OG is not None:
    nsteps_mode = num_steps(norm_mode, req_error, total_time)
    print(f'{nsteps_mode} Trotter steps are required for a {100*req_error}% error tolerance in simulation of the hamiltonian using the mode based fragmentation scheme.')

112843 Trotter steps are required for a 1.0% error tolerance in simulation of the hamiltonian using the OG fragmentation scheme.
112843 Trotter steps are required for a 1.0% error tolerance in simulation of the hamiltonian using the mode based fragmentation scheme.


### Initial State Preparation

Before the time evolution can begin, the quantum computer must be prepared in a specific initial state. This involves preparing both the resource states required for arithmetic operations and the physical state of the molecular system. The following cells estimate the cost of these preparation routines.

#### Preparing a resource state used for the phase gradient operations:

In [33]:
def phasegrad_circ(b, phase_grad_wires):
    re.ResourcePhaseGradient(num_wires=b, wires=phase_grad_wires)

# We change the default single qubit rotation precision to 1e-15 so that the error
# from the roations can be neglected for error in the phase added.
epsilon = 1e-12

res_initial = re.estimate_resources(
    phasegrad_circ, 
    single_qubit_rotation_error=epsilon,)(b, phase_grad_wires)

#### Preparing the mode registers in an Harmonic Oscillator state:

In [34]:
def stateprep_circ(k, mode_wires):
    re.ResourceQROMStatePreparation(num_state_qubits=k, wires=mode_wires)

res_initial += M*re.estimate_resources(stateprep_circ)(k, mode_wires)
print(res_initial)

--- Resources: ---
 Total qubits: 33
 Total gates : 2.230E+4
 Qubit breakdown:
  clean qubits: 13, dirty qubits: 0, algorithmic qubits: 20
 Gate breakdown:
  {'Z': 1, 'S': 1, 'T': 1.438E+4, 'X': 1.920E+3, 'CNOT': 2.352E+3, 'Toffoli': 864, 'Hadamard': 2.784E+3}


## Calculating the total algorithm cost:

#### For the OG Fragmentation scheme:

In [37]:
if norm_OG is not None:
    res_total_OG = res_initial + step_OG_res*nsteps_OG
    print_Qubit_Toff(res_total_OG)

Qubit Count = 166
Toffoli Count = 3.563E+09


#### For the mode based fragmentation scheme:

In [38]:
if norm_OG is not None:
    res_total_mode = res_initial + step_mode_res*nsteps_mode
    print_Qubit_Toff(res_total_mode)

Qubit Count = 166
Toffoli Count = 2.997E+09
